### Training MMIDAS - a coupled mixture VAE model
This notebook guides you through the process of training a mixture variational autoencoder.

In [1]:
import mmidas
from mmidas.nn_model import mixVAEConfig, loss_fn
from mmidas.cpl_mixvae import cpl_mixVAE
from mmidas.utils.tools import get_paths
from mmidas.utils.dataloader import load_data, get_loaders
import importlib
import os
import torch as t

import warnings
warnings.filterwarnings('ignore')

In [33]:
t.cuda.current_device()

0

In [2]:
t.cuda.is_available()
t.cuda.device_count()

4

In [3]:
t.cuda.get_device_name()

'NVIDIA A100-PCIE-40GB'

: 

Specify the training parameters.

In [3]:
n_categories = 120 # upper bound of number of categories (clusters)
state_dim = 2 # continuous (state) variable dimensionality 
n_arm = 2 # number of arms
latent_dim = 10 # latent dimensionality of the model
batch_size = 5000 # mini-batch size for training
n_epoch = 10 # number of epochs for training
n_epoch_p = 5 # number of epochs for pruning
min_con = 0.9 # minimum consensus among arms
max_prun_it = 2 # maximum number of pruning iterations
batch_size = 5000 # mini-batch size for training
lr = 1e-3 # learning rate for training

Load the prepared data (as described in ```1_data_prep.ipynb```) and create training and validation sets.

In [4]:
toml_file = 'pyproject.toml'
sub_file = 'smartseq_files'
config = get_paths(toml_file=toml_file, sub_file=sub_file)
data_path = config['paths']['main_dir'] / config['paths']['data_path']
data_file = data_path / config[sub_file]['anndata_file']

/allen/programs/celltypes/workgroups/mousecelltypes/Hilal/MMIDAS/pyproject.toml
Getting files directories belong to smartseq_files...


In [5]:
data = load_data(datafile=data_file)
trainloader, testloader, _, = get_loaders(dataset=data['log1p'], batch_size=batch_size)

data is loaded!
 --------- Data Summary --------- 
num cell types: 115, num cells: 22365, num genes:5032


Create a designated folder to store training files

In [6]:
n_run = 1
augmentation = False
folder_name = f'run_{n_run}_K_{n_categories}_Sdim_{state_dim}_aug_{augmentation}_lr_{lr}_n_arm_{n_arm}_nbatch_{batch_size}' + \
            f'_nepoch_{n_epoch}_nepochP_{n_epoch_p}'
saving_folder = config['paths']['main_dir'] / config['paths']['saving_path']
saving_folder = saving_folder / folder_name
os.makedirs(saving_folder, exist_ok=True)
os.makedirs(saving_folder / 'model', exist_ok=True)
saving_folder = str(saving_folder)

Construct a cpl-mixVAE object and launch its training on the prepared data.

In [31]:
importlib.reload(mmidas.cpl_mixvae)
importlib.reload(mmidas.nn_model)
from mmidas.cpl_mixvae import cpl_mixVAE

cplMixVAE = cpl_mixVAE(saving_folder=saving_folder, device='cuda', seed=546)
cplMixVAE.init(categories=n_categories,
                     state_dim=state_dim,
                     input_dim=data['log1p'].shape[1],
                     lowD_dim=latent_dim,
                     lr=lr,
                     arms=n_arm)

model_file = cplMixVAE.train(train_loader=trainloader,
                             test_loader=testloader,
                             n_epoch=n_epoch,
                             n_epoch_p=n_epoch_p,
                             min_con=min_con,
                             max_prun_it=max_prun_it)

device: NVIDIA A100-PCIE-40GB
Start training ...


 10%|█         | 1/10 [00:00<00:06,  1.39it/s, Total Loss=1.7e+8, Rec_arm_1=8.8, Rec_arm_2=8.8, Joint Loss=1.7e+8, Entropy=-6.88, Distance=0.508, Elapsed Time=0.718, Validation Loss=2.67e+10, Validation Rec. Loss=8.77]

           -- EPOCH 0 --           
Total Loss:                    169748142.0000                
Rec_arm_1:                     8.7996                        
Rec_arm_2:                     8.7953                        
Joint Loss:                    169659600.0000                
Entropy:                       -6.8828                       
Distance:                      0.5078                        
Elapsed Time:                  0.7175                        
Validation Loss:               26710128640.0000              
Validation Rec. Loss:          8.7692                        


 20%|██        | 2/10 [00:01<00:04,  1.81it/s, Total Loss=6.55e+7, Rec_arm_1=8.75, Rec_arm_2=8.74, Joint Loss=6.54e+7, Entropy=-6.96, Distance=0.486, Elapsed Time=0.435, Validation Loss=2.86e+10, Validation Rec. Loss=8.69]

           -- EPOCH 1 --           
Total Loss:                    65482671.0000                 
Rec_arm_1:                     8.7534                        
Rec_arm_2:                     8.7366                        
Joint Loss:                    65394656.0000                 
Entropy:                       -6.9614                       
Distance:                      0.4856                        
Elapsed Time:                  0.4350                        
Validation Loss:               28589053952.0000              
Validation Rec. Loss:          8.6938                        


...

 80%|████████  | 4/5 [00:02<00:00,  1.93it/s, Total Loss=2.77e+6, Rec_arm_1=3.12, Rec_arm_2=3.37, Joint Loss=2.73e+6, Entropy=-7.67, Distance=0.404, Elapsed Time=0.454, Validation Loss=2.47e+10, Validation Rec. Loss=3.24]

           -- EPOCH 3 --           
Total Loss:                    2767516.3125                  
Rec_arm_1:                     3.1211                        
Rec_arm_2:                     3.3730                        
Joint Loss:                    2734837.5000                  
Entropy:                       -7.6743                       
Distance:                      0.4038                        
Elapsed Time:                  0.4542                        
Validation Loss:               24659744768.0000              
Validation Rec. Loss:          3.2373                        


100%|██████████| 5/5 [00:02<00:00,  1.83it/s, Total Loss=2.72e+6, Rec_arm_1=3.11, Rec_arm_2=3.36, Joint Loss=2.69e+6, Entropy=-7.69, Distance=0.402, Elapsed Time=0.677, Validation Loss=2.32e+10, Validation Rec. Loss=3.23]

           -- EPOCH 4 --           
Total Loss:                    2724886.8750                  
Rec_arm_1:                     3.1126                        
Rec_arm_2:                     3.3619                        
Joint Loss:                    2692306.7500                  
Entropy:                       -7.6895                       
Distance:                      0.4022                        
Elapsed Time:                  0.6765                        
Validation Loss:               23217799168.0000              
Validation Rec. Loss:          3.2327                        


No more pruning!
Training is done!


In [14]:
x = t.zeros(1)

x += t.tensor([5])

x

tensor([5.])

In [32]:
importlib.reload(mmidas.cpl_mixvae)
importlib.reload(mmidas.nn_model)
from mmidas.cpl_mixvae import cpl_mixVAE

cplMixVAE = cpl_mixVAE(saving_folder=saving_folder, device='cuda', seed=546)
cplMixVAE.init(categories=n_categories,
                     state_dim=state_dim,
                     input_dim=data['log1p'].shape[1],
                     lowD_dim=latent_dim,
                     lr=lr,
                     arms=n_arm,)
fsdp_dct = cplMixVAE._fsdp(train_loader=trainloader,
                           val_loader=testloader,
                           epochs=n_epoch,
                           n_epoch_p=n_epoch_p,
                           min_con=min_con,
                           max_prun_it=max_prun_it,
                           device=cplMixVAE.device,
                           model=cplMixVAE.model,
                           opt=cplMixVAE.optimizer,
                           rank=cplMixVAE.device)

device: NVIDIA A100-PCIE-40GB


 10%|█         | 1/10 [00:00<00:07,  1.25it/s, Total Loss=tensor(1.6975e+08, device='cuda:0'), Rec_arm_1=tensor(8.7996, device='cuda:0'), Rec_arm_2=tensor(8.7953, device='cuda:0'), Joint Loss=tensor(1.6966e+08, device='cuda:0'), Entropy=tensor(-6.8828, device='cuda:0'), Distance=tensor(1.6966e+08, device='cuda:0')]

           -- EPOCH 0 --           
Total Loss:                    169748144.0000                
Rec_arm_1:                     8.7996                        
Rec_arm_2:                     8.7953                        
Joint Loss:                    169659600.0000                
Entropy:                       -6.8828                       
Distance:                      169659488.0000                


 20%|██        | 2/10 [00:01<00:04,  1.65it/s, Total Loss=tensor(65578188., device='cuda:0'), Rec_arm_1=tensor(8.7534, device='cuda:0'), Rec_arm_2=tensor(8.7366, device='cuda:0'), Joint Loss=tensor(65490176., device='cuda:0'), Entropy=tensor(-6.9559, device='cuda:0'), Distance=tensor(65490076., device='cuda:0')]   

           -- EPOCH 1 --           
Total Loss:                    65578188.0000                 
Rec_arm_1:                     8.7534                        
Rec_arm_2:                     8.7366                        
Joint Loss:                    65490176.0000                 
Entropy:                       -6.9559                       
Distance:                      65490076.0000                 


...

           -- EPOCH 7 --           
Total Loss:                    7336511.0000                  
Rec_arm_1:                     5.6224                        
Rec_arm_2:                     4.9703                        
Joint Loss:                    7283208.5000                  
Entropy:                       -7.4925                       
Distance:                      7283106.0000                  


 90%|█████████ | 9/10 [00:05<00:00,  1.72it/s, Total Loss=tensor(5922421., device='cuda:0'), Rec_arm_1=tensor(4.4031, device='cuda:0'), Rec_arm_2=tensor(4.2988, device='cuda:0'), Joint Loss=tensor(5878633., device='cuda:0'), Entropy=tensor(-7.5280, device='cuda:0'), Distance=tensor(5878530., device='cuda:0')]    

           -- EPOCH 8 --           
Total Loss:                    5922421.0000                  
Rec_arm_1:                     4.4031                        
Rec_arm_2:                     4.2988                        
Joint Loss:                    5878633.0000                  
Entropy:                       -7.5280                       
Distance:                      5878530.0000                  


100%|██████████| 10/10 [00:05<00:00,  1.76it/s, Total Loss=tensor(5494302.5000, device='cuda:0'), Rec_arm_1=tensor(4.0783, device='cuda:0'), Rec_arm_2=tensor(4.2416, device='cuda:0'), Joint Loss=tensor(5452436.5000, device='cuda:0'), Entropy=tensor(-7.5488, device='cuda:0'), Distance=tensor(5452334., device='cuda:0')]

           -- EPOCH 9 --           
Total Loss:                    5494302.5000                  
Rec_arm_1:                     4.0783                        
Rec_arm_2:                     4.2416                        
Joint Loss:                    5452436.5000                  
Entropy:                       -7.5488                       
Distance:                      5452334.0000                  


In [11]:
# Start training ...
# ====> Epoch:0, Total Loss: 137667456.0000, Rec_arm_1: 8.7999, Rec_arm_2: 8.7966, Joint Loss: 137578912.0000, Entropy: -7.1332, Distance: 0.4805, Elapsed Time:4.13
# ====> Validation Total Loss: 28064712704.0000, Rec. Loss: 8.7721
# ====> Epoch:1, Total Loss: 69825152.0000, Rec_arm_1: 8.7497, Rec_arm_2: 8.7451, Joint Loss: 69737120.0000, Entropy: -7.3747, Distance: 0.4512, Elapsed Time:3.64
# ====> Validation Total Loss: 25941938176.0000, Rec. Loss: 8.6993
# ====> Epoch:2, Total Loss: 42043478.0000, Rec_arm_1: 8.6571, Rec_arm_2: 8.6533, Joint Loss: 41956372.0000, Entropy: -7.3914, Distance: 0.4467, Elapsed Time:3.58
# ====> Validation Total Loss: 22861494272.0000, Rec. Loss: 8.5640
# ====> Epoch:3, Total Loss: 26674182.0000, Rec_arm_1: 8.4823, Rec_arm_2: 8.4839, Joint Loss: 26588808.0000, Entropy: -7.5388, Distance: 0.4278, Elapsed Time:3.60
# ====> Validation Total Loss: 20230934528.0000, Rec. Loss: 8.3146
# ====> Epoch:4, Total Loss: 17169558.2500, Rec_arm_1: 8.1604, Rec_arm_2: 8.1765, Joint Loss: 17087350.0000, Entropy: -7.6280, Distance: 0.4243, Elapsed Time:3.65
# ====> Validation Total Loss: 20135974912.0000, Rec. Loss: 7.8776
# ====> Epoch:5, Total Loss: 11332364.7500, Rec_arm_1: 7.5881, Rec_arm_2: 7.6376, Joint Loss: 11255748.0000, Entropy: -7.7191, Distance: 0.4143, Elapsed Time:3.53
# ====> Validation Total Loss: 23999512576.0000, Rec. Loss: 7.1436
# ====> Epoch:6, Total Loss: 8132679.8750, Rec_arm_1: 6.6561, Rec_arm_2: 6.7642, Joint Loss: 8065148.5000, Entropy: -7.7605, Distance: 0.4065, Elapsed Time:3.60
# ====> Validation Total Loss: 31686617088.0000, Rec. Loss: 6.0345
# ====> Epoch:7, Total Loss: 6335305.1250, Rec_arm_1: 5.3905, Rec_arm_2: 5.5418, Joint Loss: 6280293.0000, Entropy: -7.8043, Distance: 0.4005, Elapsed Time:3.63
# ====> Validation Total Loss: 36792733696.0000, Rec. Loss: 4.7336
# ====> Epoch:8, Total Loss: 5184599.3750, Rec_arm_1: 4.3659, Rec_arm_2: 4.3467, Joint Loss: 5140757.0000, Entropy: -7.8469, Distance: 0.3987, Elapsed Time:3.51
# ====> Validation Total Loss: 38023708672.0000, Rec. Loss: 4.0776
# ====> Epoch:9, Total Loss: 4790232.1250, Rec_arm_1: 4.2830, Rec_arm_2: 4.0376, Joint Loss: 4748362.0000, Entropy: -7.8604, Distance: 0.3998, Elapsed Time:3.57
# ====> Validation Total Loss: 36604887040.0000, Rec. Loss: 3.9312
# Training with pruning...
# Purned categories: [1]
# ====> Epoch:0, Total Loss: 4001048.3125, Rec_arm_1: 3.8611, Rec_arm_2: 3.8509, Joint Loss: 3962241.0000, Entropy: -7.8664, Distance: 0.3989, Elapsed Time:3.85
# ====> Validation Total Loss: nan, Rec. Loss: 3.5026
# ====> Epoch:1, Total Loss: 3639977.4375, Rec_arm_1: 3.5320, Rec_arm_2: 3.3895, Joint Loss: 3605148.2500, Entropy: -7.8894, Distance: 0.3974, Elapsed Time:3.83
# ====> Validation Total Loss: nan, Rec. Loss: 3.4378
# ====> Epoch:2, Total Loss: 3534881.8125, Rec_arm_1: 3.4955, Rec_arm_2: 3.3030, Joint Loss: 3500671.7500, Entropy: -7.9030, Distance: 0.3960, Elapsed Time:4.10
# ====> Validation Total Loss: nan, Rec. Loss: 3.4195
# ====> Epoch:3, Total Loss: 3155906.8125, Rec_arm_1: 3.4152, Rec_arm_2: 3.2708, Joint Loss: 3122262.7500, Entropy: -7.9187, Distance: 0.3939, Elapsed Time:4.40
# ====> Validation Total Loss: nan, Rec. Loss: 3.3164
# ====> Epoch:4, Total Loss: 2953709.7500, Rec_arm_1: 3.3518, Rec_arm_2: 3.1916, Joint Loss: 2920782.5000, Entropy: -7.9333, Distance: 0.3947, Elapsed Time:3.98
# ====> Validation Total Loss: nan, Rec. Loss: 3.2596
# Training with pruning...
# Purned categories: [1 2]
# ====> Epoch:0, Total Loss: 2872254.8750, Rec_arm_1: 3.3458, Rec_arm_2: 3.1647, Joint Loss: 2839493.0000, Entropy: -7.9148, Distance: 0.3927, Elapsed Time:3.81
# ====> Validation Total Loss: nan, Rec. Loss: 3.2327
# ====> Epoch:1, Total Loss: 2638138.1875, Rec_arm_1: 3.3082, Rec_arm_2: 3.1392, Joint Loss: 2605694.2500, Entropy: -7.9227, Distance: 0.3921, Elapsed Time:3.87
# ====> Validation Total Loss: nan, Rec. Loss: 3.1979
# ====> Epoch:2, Total Loss: 2455072.6250, Rec_arm_1: 3.2809, Rec_arm_2: 3.0947, Joint Loss: 2422990.5000, Entropy: -7.9277, Distance: 0.3917, Elapsed Time:3.78
# ====> Validation Total Loss: nan, Rec. Loss: 3.1861
# ====> Epoch:3, Total Loss: 2434174.7500, Rec_arm_1: 3.2733, Rec_arm_2: 3.0776, Joint Loss: 2402216.5000, Entropy: -7.9398, Distance: 0.3898, Elapsed Time:3.97
# ====> Validation Total Loss: nan, Rec. Loss: 3.1811
# ====> Epoch:4, Total Loss: 2520542.7500, Rec_arm_1: 3.2612, Rec_arm_2: 3.0682, Joint Loss: 2488693.0000, Entropy: -7.9395, Distance: 0.3899, Elapsed Time:3.85
# ====> Validation Total Loss: nan, Rec. Loss: 3.1711
# No more pruning!
# Training is done!

In [12]:
def join(s, *args, **kwargs): 
    return s.join(*args, **kwargs)

join('', ['a', 'b', 'c'])

'abc'

Working directly with command line, you have the option to train the model using a Python file, such as ```tutorial/train_unimodal.py``` as follows.

```
python train_unimodal.py --n_epoch 10 --n_epoch_p 5 --max_prun_it 2
```
or
```
python train_unimodal.py --n_epoch 10 --n_epoch_p 5 --max_prun_it 2 --device 'cuda'
```